In [1]:
# https://resources.oreilly.com/examples/9780596529321/tree/master

class crawler:
    # Initialize the crawler with the name of database.
    def __init__(self, db):
        pass
    
    def __del__(self):
        pass

    def commit(self):
        pass
    
    # Auxialliary function for getting an entry id and adding it if it is not present.
    def get_entry_id(self, table, field, value, create_new = True):
        return None
    
    # Index an individual page.
    def add_to_index(self, url, soup):
        print(f'indexing {url}')
    
    # Extract the text from the HTML page (no tags).
    def get_text_only(self, soup):
        return None
    
    # Separate the words by non-whitespace character.
    def separate_words(self, text):
        return None
    
    # Return true if this url is already indexed.
    def is_indexed(self, url):
        return False
    
    # Add a link between pages.
    def add_link_ref(self, url_from, url_to, link_text):
        pass

    # Starting with a list of pages, do a breadth first search to the given depth,
    # indexing pages as we go.
    def crawl(self, pages, depth = 2):
        pass
    
    # Create the database tables.
    def create_index_tables(self):
        pass

In [5]:
import urllib3
http = urllib3.PoolManager()
url = 'https://en.wikipedia.org/wiki/Programming_language'
# 'http://kiwitobes.com/wiki/Programming_language.html'
r = http.request('GET', url)
r.status, r.data[:500]

/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(200,
 b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Programming language - Wikipedia</title>\n<script>document.documentElement.className=document.documentElement.className.replace(/(^|\\s)client-nojs(\\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Programming_language","wgTitle":"Programming language","wgCurRevisionId":900725449,"wgRevisionId":900725449,"wgArticleId":23015')

In [11]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [82]:
import re
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import sqlite3

http = urllib3.PoolManager()
url = 'https://en.wikipedia.org/wiki/Programming_language'
# pip3 install beautifulsoup4

class Crawler:
    # Initialize the crawler with the name of database.
    def __init__(self, db):
        self.conn = sqlite3.connect(db)
    
    def __del__(self):
        self.conn.close()

    def commit(self):
        self.conn.commit()
    
    # Auxialliary function for getting an entry id and adding it if it is not present.
    def get_entry_id(self, table, field, value, create_new = True):
        c = self.conn.cursor()
        c.execute(f'select rowid from {table} where ? = ?', (field, value))
        res = c.fetchone()
        if res == None:
            r = self.conn.execute(f'insert into {table} ({field}) values (?)', (value,))
            return r.lastrowid
        else:
            return res[0]
    
    # Index an individual page.
    def add_to_index(self, url, soup):
        if self.is_indexed(url): return
        print(f'indexing {url}')
        
        # Get the individual words.
        text = self.get_text_only(soup)
        words = self.separate_words(text)
        
        # Get the URL id.
        url_id = self.get_entry_id('urllist', 'url', url)
        
        # Link each word to this url.
        for i in range(len(words)):
            word = words[i]
            if word in stopwords: continue
            word_id = self.get_entry_id('wordlist', 'word', word)
            self.conn.execute('insert into wordlocation(url_id, word_id,location) values (?, ?, ?)', (url_id, word_id, i))
    
    # Extract the text from the HTML page (no tags).
    def get_text_only(self, soup):
        v = soup.string
        if v == None:
            c = soup.contents
            result_text = ''
            for t in c:
                subtext = self.get_text_only(t)
                result_text += subtext + '\n'
            return result_text
        else:
            return v.strip()
    
    # Separate the words by non-whitespace character.
    def separate_words(self, text):
        splitter = re.compile('\\W*')
        return [s.lower() for s in splitter.split(text) if s != '']
    
    # Return true if this url is already indexed.
    def is_indexed(self, url):
        cur = self.conn.cursor()
        res = cur.execute('select url from urllist where url = ?', (url,)).fetchone()
        if res != None:
            # Check if it has actually been crawled.
            cur = self.conn.cursor()
            print('got cursor', cur, res[0])
            c = cur.execute('select * from wordlocation where url_id = ?', (res[0],))
            if c != None:
                res = c.fetchone()
                if res != None: return True
        return False
    
    # Add a link between pages.
    def add_link_ref(self, url_from, url_to, link_text):
        pass

    # Starting with a list of pages, do a breadth first search to the given depth,
    # indexing pages as we go.
    def crawl(self, pages, depth = 2):
        for i in range(depth):
            new_pages = set()
            for page in pages:
                try:
                    r = http.request('GET', page)
                    # r.status, r.data 
                    print(f'loaded page {page} {r.status}')
                except:
                    print(f'could not open page {page}')
                    continue
                soup = BeautifulSoup(r.data, 'html.parser')
                self.add_to_index(page, soup)
                
                links = soup('a')
                for link in links:
                    print('getting all links', link, page)
                    if not 'href' in link: continue
                    url = urljoin(page, link['href'])
                    print('joined url is', url)
                    if url.find("'") != -1: continue
                    url = url.split('#')[0] # Remove location portion.
                    if url[0:4] == 'http' and not self.is_indexed(url):
                        new_pages.add(url)
                    link_text = self.get_text_only(link)
                    self.add_link_ref(page, url, link_text)
                self.commit()
            pages = new_pages
                
    # Create the database tables.
    def create_index_tables(self):
        self.conn.execute('create table if not exists urllist(url)')
        self.conn.execute('create table if not exists wordlist(word)')
        self.conn.execute('create table if not exists wordlocation(url_id, word_id, location)')
        self.conn.execute('create table if not exists link(from_id integer, to_id integer)')
        self.conn.execute('create table if not exists linkwords(word_id, link_id)')
        self.conn.execute('create index if not exists wordidx on wordlist(word)')
        self.conn.execute('create index if not exists urlidx on urllist(url)')
        self.conn.execute('create index if not exists wordurlidx on wordlocation(word_id)')
        self.conn.execute('create index if not exists urltoidx on link(to_id)')
        self.conn.execute('create index if not exists urlfromidx on link(from_id)')
        self.commit()

In [83]:
# Crawler('').crawl(['https://en.wikipedia.org/wiki/Perl'])

In [84]:
crawler = Crawler('searchindex.db')
crawler.create_index_tables()
crawler.crawl(['https://en.wikipedia.org/wiki/Perl'])

/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


loaded page https://en.wikipedia.org/wiki/Perl 200
indexing https://en.wikipedia.org/wiki/Perl
getting all links <a id="top"></a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/wiki/Wikipedia:Good_articles" title="This is a good article. Follow the link for more information."><img alt="This is a good article. Follow the link for more information." data-file-height="185" data-file-width="180" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/9/94/Symbol_support_vote.svg/19px-Symbol_support_vote.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/9/94/Symbol_support_vote.svg/29px-Symbol_support_vote.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/9/94/Symbol_support_vote.svg/39px-Symbol_support_vote.svg.png 2x" width="19"/></a> https://en.wikipedia.org/wiki/Perl
getting all links <a class="mw-jump-link" href="#mw-head">Jump to navigation</a> https://en.wikipedia.org/wiki/Perl
getting all links <a class="mw-jump-link" href="#p-searc

getting all links <a href="/wiki/World_Wide_Web" title="World Wide Web">Web</a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/wiki/ActiveState" title="ActiveState">ActivePerl</a> https://en.wikipedia.org/wiki/Perl
getting all links <a class="mw-redirect" href="/wiki/ISAPI" title="ISAPI">ISAPI</a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/wiki/Bytecode" title="Bytecode">bytecode</a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/w/index.php?title=Perl&amp;action=edit&amp;section=19" title="Edit section: Optimizing">edit</a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/wiki/Interpreted_language#Disadvantages" title="Interpreted language">give problems</a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/wiki/C_(programming_language)" title="C (programming language)">C</a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/wiki/XS_(Perl)" title="XS (Perl)">XS</a> https://en.wikipedia.org/wiki

getting all links <a href="/wiki/Template:FOSS" title="Template:FOSS"><abbr style=";;background:none transparent;border:none;-moz-box-shadow:none;-webkit-box-shadow:none;box-shadow:none; padding:0;" title="View this template">v</abbr></a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/wiki/Template_talk:FOSS" title="Template talk:FOSS"><abbr style=";;background:none transparent;border:none;-moz-box-shadow:none;-webkit-box-shadow:none;box-shadow:none; padding:0;" title="Discuss this template">t</abbr></a> https://en.wikipedia.org/wiki/Perl
getting all links <a class="external text" href="//en.wikipedia.org/w/index.php?title=Template:FOSS&amp;action=edit"><abbr style=";;background:none transparent;border:none;-moz-box-shadow:none;-webkit-box-shadow:none;box-shadow:none; padding:0;" title="Edit this template">e</abbr></a> https://en.wikipedia.org/wiki/Perl
getting all links <a href="/wiki/Free_and_open-source_software" title="Free and open-source software">Free and open-so

getting all links <a class="interlanguage-link-target" href="https://fa.wikipedia.org/wiki/%D9%BE%D8%B1%D9%84" hreflang="fa" lang="fa" title="پرل – Persian">فارسی</a> https://en.wikipedia.org/wiki/Perl
getting all links <a class="interlanguage-link-target" href="https://fr.wikipedia.org/wiki/Perl_(langage)" hreflang="fr" lang="fr" title="Perl (langage) – French">Français</a> https://en.wikipedia.org/wiki/Perl
getting all links <a class="interlanguage-link-target" href="https://ga.wikipedia.org/wiki/Perl" hreflang="ga" lang="ga" title="Perl – Irish">Gaeilge</a> https://en.wikipedia.org/wiki/Perl
getting all links <a class="interlanguage-link-target" href="https://gl.wikipedia.org/wiki/Perl" hreflang="gl" lang="gl" title="Perl – Galician">Galego</a> https://en.wikipedia.org/wiki/Perl
getting all links <a class="interlanguage-link-target" href="https://gu.wikipedia.org/wiki/%E0%AA%AA%E0%AA%B0%E0%AB%8D%E0%AA%B2" hreflang="gu" lang="gu" title="પર્લ – Gujarati">ગુજરાતી</a> https://en.wikiped

In [93]:
[row for row in crawler.conn.execute('select rowid from wordlocation where word_id = 1')]

[(1,)]